# 文章目录

[TCP/IP](#1)
- TCP编程
- UDP编程

[电子邮件](#2)

# TCP/IP
<a id='1'></a>

###  TCP编程

Socket是网络编程的一个抽象概念。通常我们用一个Socket表示打开了一个网络链接，而打开一个Socket需要知道目标计算机的IP地址和端口号，再指定协议类型即可


### 客户端

大多数链接都是可靠的TCP链接。创建TCP连接时，主动发起连接的叫客户端，被动响应连接的叫服务器。

举个例子，当我们再浏览器中访问新浪时，我们自己的计算器就是客户端，浏览器会主动向新浪的服务器发起连接。如果一切顺利，新浪的服务器接受了我们的连接，一个TCP连接就建立起来的，后面的通信就是发送网页内容了。

所以，我们要创建一个基于TCP连接的Socket，可以这样做：

In [2]:
#导入socket库
import socket

In [13]:
#创建一个socket
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#建立连接
s.connect(('www.sina.com.cn', 80))

创建Socket时，AF_INET指定使用IPV4协议，如果要用更先进的IPV6协议要指定为AF_INET6。SOCK_STREAM指定使用面向流的TCP协议，这样一个Socket对象就创建成功，但是还没有建立连接

客户端要主动发起TCP连接，必须知道服务器的IP地址和端口号。新浪网站的IP地址可以用域名www.sina.com.cn 自动转换到IP地址，但是怎么知道新浪服务器的端口号呢？

作为服务器，提供什么样的服务，端口号就必须固定下来。由于我们想要访问网页，因此新浪提供网页服务的服务器必须把端口号固定在80端口，因为80端口时Web服务的标准端口。其他服务都有对应的标准端口号，例如SMTP服务时25端口，FTP服务时21端口，等等。端口号小于1024的时Internet标准服务的端口，端口号大于1024的可以任意使用

建立TCP连接后，我们就可以向新浪服务器发送请求，要求返回首页内容：

In [14]:
s.send('GET / HTTP/1.1\r\nHost: www.sina.com.cn\r\nConnection: close\r\n\r\n')

60

TCP连接创建的是双向通道，双方都可以同时给对方发数据。但是谁先发谁后发，怎么协调，要根据具体的协议来决定。例如HTTP协议规定客户端必须先发请求给服务器，服务器收到后才发数据给客户端。

发送的文本格式必须符合HTTP标准，如果格式没问题，接下来就可以接受新浪服务器返回的数据了：

In [15]:
buffer = []
while True:
    #每次接受1k字节：
    d = s.recv(1024)
    if(d):
        buffer.append(d)
    else:
        break
data = ''.join(buffer)
print data

HTTP/1.1 200 OK
Content-Type: text/html
Vary: Accept-Encoding
X-Powered-By: shci_v1.03
Server: nginx
Date: Sat, 08 Apr 2017 09:13:50 GMT
Last-Modified: Sat, 08 Apr 2017 09:12:08 GMT
Expires: Sat, 08 Apr 2017 09:14:50 GMT
Cache-Control: max-age=60
Age: 9
Content-Length: 598838
X-Cache: HIT from ctc.guangzhou.sinacache.45.nb.sinaedge.com
Connection: close

<!DOCTYPE html>
<!-- [ published at 2017-04-08 17:12:05 ] -->
<html>
<head>
    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge" />
    <title>新浪首页</title>
	<meta name="keywords" content="新浪,新浪网,SINA,sina,sina.com.cn,新浪首页,门户,资讯" />
	<meta name="description" content="新浪网为全球用户24小时提供全面及时的中文资讯，内容覆盖国内外突发新闻事件、体坛赛事、娱乐时尚、产业资讯、实用信息等，设有新闻、体育、娱乐、财经、科技、房产、汽车等30多个内容频道，同时开设博客、视频、论坛等自由互动交流空间。" />
    <link rel="mask-icon" sizes="any" href="//www.sina.com.cn/favicon.svg" color="red">
	<meta name="stencil" content="PGLS000022" />
	<meta name="publishid" content="30,131,1" />


接收数据时，调用recv(max)方法，一次最多接收指定的字节数，因此，在一个while循环中反复接收，直到recv()返回空数据，表示接收完毕，退出循环。

当我们接收完数据后，调用close()方法关闭Socket，这样，一次完整的网络通信就结束了：

In [16]:
s.close()

接受到的数据包包括HTTP头和网页本身，我们只需要把HTTP头和网页分离一下，把HTTP头打印出来，网页内容保存到文件：

In [17]:
header, html = data.split('\r\n\r\n', 1)
print header
# 把接收的数据写入文件:
with open('sina.html', 'wb') as f:
    f.write(html)

HTTP/1.1 200 OK
Content-Type: text/html
Vary: Accept-Encoding
X-Powered-By: shci_v1.03
Server: nginx
Date: Sat, 08 Apr 2017 09:13:50 GMT
Last-Modified: Sat, 08 Apr 2017 09:12:08 GMT
Expires: Sat, 08 Apr 2017 09:14:50 GMT
Cache-Control: max-age=60
Age: 9
Content-Length: 598838
X-Cache: HIT from ctc.guangzhou.sinacache.45.nb.sinaedge.com
Connection: close


### 服务器

和客户端编程相比，服务器编程更复杂些

服务器进程首先要绑定一个端口并监听来自其他客户端的连接。如果某个客户端连接过来了，服务器就与该客户端建立Socket连接，随后的通信就靠这个Socket连接了。

所以，服务器会打开固定端口（比如80）监听，每来一个客户端连接，就创建该Socket连接。由于服务器会有大量来自客户端的连接，所以服务器要能够区分一个Socket连接时和哪个客户端绑定的。一个Socket依赖4项：
- 服务器地址
- 服务器端口
- 客户端地址
- 客户端端口
来唯一确定一个Socket

但是服务器还需要同时响应多个客户端的请求，所以每个连接都需要一个新的进程或者新的线程来处理，否则，服务器一次就只能服务一个客户端了

我们编写一个简单的服务器程序，让它接受客户端连接，把客户端发过来的字符串加上hello再发回去

In [18]:
#首先创建一个基于IPV4和TCP协议的Socket
s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)

然后我们要绑定监听的地址和端口。服务器可能有多块网卡，可以绑定到某一块网卡的IP地址上，也可以用 0.0.0.0 绑定到所有的网络地址，还可以用 127.0.0.1绑定到本机地址

127.0.0.1是个特殊的IP地址，表示本机地址，如果绑定到这个地址，客户端必须同事再本机运行才能连接，也就是说外部的计算机无法连接进来

端口号需要预先设定，因为我们写的这个服务不是标准服务，所以使用9999这个端口号

小于1024的端口号必须要有管理员权限才能绑定


In [ ]:
#监听端口
s.bind(('17.0.0.1', 9999))
#调用listen()方法开始监听端口，传入的参数指定等待连接的最大数量：
s.listen(5)
print 'Waiting for connection...'
#一个永久循环来接受连接，accept()会等待并返回一个客户端的连接：
while True:
    #接受一个新连接：
    sock, addr = s.accept()
    #创建新线程来处理TCP连接
    t = threading.Thread(target=tcplink, args=(sock, addr))
    t.start()

每个连接都必须创建新线程或者进程来处理，否则单线程再处理连接的过程中无法接受其他客户端的连接

In [19]:
def tcplink(sock, addr):
    print 'Accept new connection form %s:%s...' %addr
    sock.send('Welcome!')
    while True:
        data = sock.recv(1024)
        time.sleep(1)
        if data == 'exit' or not data:
            break
        sock.send('Hello, %s!' %data)
    sock.close()
    
    print 'Connection from %s:%s closed.' %addr

In [ ]:
#客户端程序
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# 建立连接:
s.connect(('127.0.0.1', 9999))
#接受欢迎消息：
print s.recv(1024)
for data in['Michael', 'Tracy', 'Sarah']:
    #发送数据：
    s.send(data)
    print s.recv(1024)
    
s.send('exit')
s.close()

### UDP 编程

TCP是建立可靠连接，并且通信双方都可以以流的形式发送数据。相对TCP，UDP则是面向无连接的协议。

使用UDP协议时，不需要建立连接，只需要知道对方的IP地址和端口号就可以直接发送哦给数据包，但是能不能到达就不知道了。

虽然UDP传输数据不可靠，但是它的有点是和TCP比，速度快，对于不要求可靠到达的数据，就可以使用UDP协议。

我们来看看如何通过UDP协议传输数据。和TCP类似，使用UDP的通信双方也分为客户端和服务器

In [3]:
import socket
#服务器首先需要绑定端口
s = socket.socket(socket.AF_INET,socket.SOCK_DGRAM)
#绑定端口：
s.bind(('127.0.0.1', 9999))

创建socket时SOCK_DGRAM指定了这个socket的类型时UDP。绑定端口和TCP一样，但是不需要调用listen()方法，而时直接接受来自任何客户端的数据：

In [ ]:
print 'Bind UDP on 9999...'
while True:
    #接收数据：
    data, addr = s.recvfrom(1024)
    print 'Received from %s:%s.' %addr
    s.sendto('Hello, %s!' %data, addr)

recvfrom()方法返回数据和客户端的地址与端口，这样服务器收到数据后，直接调用sendto()就可以把数据用UDP发给客户端。

客户端使用UDP时，首先任然创建基于UDP的Socket，然后，不需要调用connct()，直接通sendto()给服务器发数据

In [ ]:
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
for data in ['Michael', 'Tracy', 'Sarah']:
    # 发送数据:
    s.sendto(data, ('127.0.0.1', 9999))
    # 接收数据:
    print s.recv(1024)
s.close()

# 电子邮件
<a id='2'></a>
Email从MUA（邮件用户代理）发送出去，不是直接到达对方电脑，而是发到MTA：Mail Transfer Agent——邮件传输代理，就是那些Email服务提供商，比如网易，新浪等等，假设我们的电子邮件是163.com，Email首先被投递到网易提供的MTA，再由网易的MTA发到对方服务商，也就是新浪的MTA。这个过程中间可能还会经过别的MTA，但是我们不关心具体路线，我们只关心速度。

Email到达新浪的MTA后，由于对方使用的是@sina.com的邮箱，因此新浪的MTA会把Email投递到邮件的最终目的地MDA：Mail Delivery Agent——邮件投递代理。 Email到达MDA后，就静静的躺在新浪的某个服务器商，存放再某个文件或者特殊的数据库里，我们将这个长期保存邮件的地方称之为电子邮箱。

和普通邮件类似，Email不会直接到达对方的电脑，因为对方电脑不一定开机，开机也不一定联网。对方要取到邮件，必须要通过MUA从MDA上把邮件取到自己的电脑上。

所以，一封电子邮件的旅程就是：
发件人 -> MUA -> MTA -> MTA -> 若干个MTA -> MDA <- MUA <- 收件人

要编写程序来发送和接收邮件，本质上就是：
- 编写MUA把邮件发送到MTA
- 编写MUA从MDA上收邮件
发邮件时，MUA和MTA使用的协议就是SMTP：Simple Mail Transfer Protocol，后面的MTA到另一个MTA也是使用SMTP协议

收到邮件后，MUA和MDA使用的协议有两种：POP：Post Office Protocol； IMAP： Internet Message Access Protocol，目前版本是4，有点时不但能取邮件，还可以直接操作MDA上存储的邮件，比如从收件箱移到垃圾箱等等。

邮件客户端软件在发邮件时，会让你先配置SMTP服务器，也就是你要发到哪个MTA上。假设你正在使用163的邮箱，你就不能直接发到新浪的MTA上，因为它只服务新浪的用户，所以，你得填163提供得SMTP服务器地址：smtp.163.com，为了证明你是163得用户，SMTP服务器还要求你填写邮箱地址和邮箱口令，这样MUA才能正常地把Email通过SMTP协议发送到MTA

类似的，从MDA收邮件时，MDA服务器也要求验证你得邮箱口令，确保不会有人冒充你收取你的邮件，所以Outlook之类的邮件客户端会要求你填写POP3或者IMAP服务器地址，邮箱地址和口令，这样，MUA才能顺利地通过POP或者IMAP协议从MDA取到邮件。


### SMTP 发送邮件 

SMTP时发送邮件的协议，Python内置对SMTP的支持，可以发送纯文本邮件、HTML邮件以及带附件的邮件。

Python对SMTP支持有smtplib和email两个模块，email负责构造邮件，smtplib负责发送邮件

In [3]:
#构造uige最简单的纯文本邮件：
from email.mime.text import MIMEText
msg = MIMEText('Hello , send by Python...', 'plain', 'utf-8')

注意到构造MIMEText对象时，第一个参数时邮件正文，第二个参数时MIME的subtype，传入‘plain最终的MIME就是’text/plain，最后一定要用utf-8编码保证多语言兼容性

In [ ]:
#输入Email地址和口令：
from_addr = raw_input('From: ')
password = raw_input('Password: ')

#输入SMTP服务器地址：
smtp_server = raw_input('SMTP server: ')

#输入收件人地址：
to_addr = raw_input('To: ')

import smtplib

server = smtplib.SMTP(smtp_server, 25) #SMTP协议默认端口为25
server.set_debuglevel(1)
server.login(from_addr, password)
server.sendmail(from_addr, [to_addr], msg.as_string())
server.quit()

我们必须把from、 to 和subject添加到MIMEText中，才是一个完整的邮件：

邮件账号密码随便填的，为了起到演示效果

In [2]:
from email import encoders
from email.header import Header
from email.mime.text import MIMEText
from email.utils import parseaddr, formataddr
import smtplib

def _format_addr(s):
    name, addr = parseaddr(s)
    return formataddr(( \
        Header(name,'utf-8').encode(), \
        addr.encode('utf-8') if isinstance(addr, unicode) else addr))

from_addr = raw_input('From: ')
password = raw_input('Password: ')
to_addr = raw_input('To: ')
smtp_server = raw_input('SMTP server: ')

msg = MIMEText('hello,send by Python...', 'plain', 'utf-8')
msg['From'] = _format_addr(u'Python爱好者<%s>' %from_addr)
msg['To'] = _format_addr(u'管理员 <%s>' %to_addr)
msg['Subject'] = Header(u'来自Python初学者的问候...', 'utf-8').encode()

server = smtplib.SMTP(smtp_server, 25)
server.set_debuglevel(1)
server.login(from_addr, password)
server.sendmail(from_addr, [to_addr], msg.as_string())
server.quit()

From: xxx@163.com
Password: 123456
To: xxxxxx@qq.com
SMTP server: smtp.163.com


send: 'ehlo [192.168.1.169]\r\n'
reply: '250-mail\r\n'
reply: '250-PIPELINING\r\n'
reply: '250-AUTH LOGIN PLAIN\r\n'
reply: '250-AUTH=LOGIN PLAIN\r\n'
reply: '250-coremail 1Uxr2xKj7kG0xkI17xGrU7I0s8FY2U3Uj8Cz28x1UUUUU7Ic2I0Y2Ur5mNYXUCa0xDrUUUUj\r\n'
reply: '250-STARTTLS\r\n'
reply: '250 8BITMIME\r\n'
reply: retcode (250); Msg: mail
PIPELINING
AUTH LOGIN PLAIN
AUTH=LOGIN PLAIN
coremail 1Uxr2xKj7kG0xkI17xGrU7I0s8FY2U3Uj8Cz28x1UUUUU7Ic2I0Y2Ur5mNYXUCa0xDrUUUUj
STARTTLS
8BITMIME
send: 'AUTH PLAIN AHh4eEAxNjMuY29tADEyMzQ1Ng==\r\n'
reply: '535 Error: authentication failed\r\n'
reply: retcode (535); Msg: Error: authentication failed


SMTPAuthenticationError: (535, 'Error: authentication failed')

我们编写了一个函数_format_addr()来格式化一个邮件地址。注意不能简单地传入name <addr@example.com>，因为如果包含中文，需要通过Header对象进行编码。
msg['To']接收的是字符串而不是list，如果有多个邮件地址，用,分隔即可。

### 发送html邮件

如果我们要发送HTML邮件，而不是普通的纯文本文件怎么办？方法很简单，在构造MIMEText对象时，把HTML字符串传进去，再把第二个参数由plain变为html就可以了：